This is going to be 3.3

# 3.3 Looking at the Lexical Vocabulary from the Perspective of the Literary Material

In section 3.2 we asked whether we can see differences between Old Babylonian literary compositions in their usage of vocabulary (lemmas and MWEs) attested in the lexical corpus. In this notebook we will change perspective and ask: are there particular lexical texts (or groups of lexical texts) that show a greater engagement with literary vocabulary than others?

In [3.1](./3_1_Lit_Lex_Vocab.ipynb) and [3.2](./3_2_Lit_Lex.ipynb) we used Multiple Word Expressions, connecting words that are found in a lexical entry by underscrores (using `MWEtokenizer()` from the nltk module). The lemmas and MWE were used visualized in Venn diagrams to illustrate the intersection between lexical and literary vocabulary.

In this notebook we will use the ngram option of the `CountVectorizer()` function in order to find sequences of lemmas that are shared between lexical and literary texts. A ngram is a continuous sequence of *n* words (or lemmas). 


In large part, this notebook uses the same techniques and the same code as section 3.2 did, and the reader is referred there for further explanation. In some aspects, however, the process is different. In particular, we will use various aspects of `CountVectorizer()` and the related function `TfidfVectorizer()` to understand the relationship in more detail.

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # this suppresses a warning about pandas from tqdm
import pandas as pd
from ipywidgets import interact
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from tqdm.auto import tqdm
tqdm.pandas() # initiate pandas support in tqdm, allowing progress_apply() and progress_map()
import zipfile
import json
import numpy as np

Open the file `lexlines.p` which was produced in [3_1_Lit_Lex_Vocab.ipynb](./3_1_Lit_Lex_Vocab.ipynb). The file contains the pickled version of the DataFrame `lex_lines` in which the lexical ([dcclt](http://oracc.org/dcclt)) corpus is represented in line-by-line format.

In [2]:
lex_lines = pd.read_pickle('output/lexlines.p')

### Special Case: OB Nippur Ura 6
The sixth chapter of the Old Babylonian Nippur version of the thematic list Ura deals with foodstuffs and drinks. This chapter was not standardized (each exemplar has its own order of items and sections) and therefore no composite text has been created in [DCCLT](http://oracc.org/dcclt). Instead, the "composite" of [OB Nippur Ura 6](http://oracc.org/dcclt/Q000043) consists of the concatenation of all known Nippur exemplars of the list of foodstuffs. In our current dataframe, therefore, there are no lines where the field `id_text` equals "Q000043".

We create a "composite" by changing the field `id_text` in all exemplars of [OB Nippur Ura 6](http://oracc.org/dcclt/Q000043) to "Q000043". 

In [3]:
Ura6 = ["dcclt/P227657",
"dcclt/P227743",
"dcclt/P227791",
"dcclt/P227799",
"dcclt/P227925",
"dcclt/P227927",
"dcclt/P227958",
"dcclt/P227967",
"dcclt/P227979",
"dcclt/P228005",
"dcclt/P228008",
"dcclt/P228200",
"dcclt/P228359",
"dcclt/P228368",
"dcclt/P228488",
"dcclt/P228553",
"dcclt/P228562",
"dcclt/P228663",
"dcclt/P228726",
"dcclt/P228831",
"dcclt/P228928",
"dcclt/P229015",
"dcclt/P229093",
"dcclt/P229119",
"dcclt/P229304",
"dcclt/P229332",
"dcclt/P229350",
"dcclt/P229351",
"dcclt/P229352",
"dcclt/P229353",
"dcclt/P229354",
"dcclt/P229356",
"dcclt/P229357",
"dcclt/P229358",
"dcclt/P229359",
"dcclt/P229360",
"dcclt/P229361",
"dcclt/P229362",
"dcclt/P229365",
"dcclt/P229366",
"dcclt/P229367",
"dcclt/P229890",
"dcclt/P229925",
"dcclt/P230066",
"dcclt/P230208",
"dcclt/P230230",
"dcclt/P230530",
"dcclt/P230586",
"dcclt/P231095",
"dcclt/P231128",
"dcclt/P231424",
"dcclt/P231446",
"dcclt/P231453",
"dcclt/P231458",
"dcclt/P231742",
"dcclt/P266520"]
lex_lines.loc[lex_lines["id_text"].isin(Ura6), "id_text"] = "dcclt/Q000043"

# Computing text length
In order to evaluate the number of matches between a lexical text and the literary corpus we need a measure of text length. Text length is defined here as the number of lemmatized words in a text.

First the lines of `lit_lines` are aggregated to lexical compositions in the DataFrame `lex_comp`. 

In [4]:
lex_comp = lex_lines.groupby(
    [lex_lines["id_text"]]).aggregate(
    {"lemma": ' '.join}).reset_index()

The function `lex_length()` computes the number of lemmas in each composition by first splitting the field `lemmas` into individual lemmas. A list comprehension removes all unlemmatized words, and the length of the resulting list is returned.

In [5]:
def lex_length(lemmas):
    lemmas = lemmas.split()
    lemmas = [lemma for lemma in lemmas if not '[na]na' in lemma] # remove unlemmatized words
    length = len(lemmas)
    return length

First add the new field `length` by calling the function `lex_length()` for every row.

The DataFrame `lex_comp` has data from all Old Babylonian lexical texts currently in [dcclt](http://oracc.org/dcclt). Not all of these texts are lemmatized. In particular, documents that have been linked to a composite text are usually not lemmatized. Such documents have no lemmatized contents and therefore have length 0. These documents are removed from `lex_comp`.

In [6]:
lex_comp['length'] = lex_comp['lemma'].progress_map(lex_length)
lex_comp = lex_comp.loc[lex_comp['length'] > 0] # remove compositions that have no lemmatized content

# Open list of Vocabulary Intersection
The file `lit_lex_vocab` is a list that includes all lemmas and Multiple Word Expressions that are shared by the literary corpus and the lexical corpus. This list was produced in [3_2_Lit_Lex.ipynb](./3_2_Lit_Lex.ipynb). In sections [3.1](./3_1_Lit_Lex_Vocab.ipynb) and [3.2](./3_2_Lit_Lex.ipynb) lexical *entries* were turned into MWEs by connecting the individual lemmas by underscores (as in `amar\[young\]n_ga\[milk\]n_gu\[eat\]v/t`). In this notebook we will take a different approach by using ngrams. For that reason we need to replace all those underscores by spaces.

This vocabulary is used in the next section for building a Document Term Matrix.

In [7]:
with open('output/lit_lex_vocab.txt', 'r', encoding = 'utf8') as l:
    lit_lex_vocab = l.read().splitlines()
lit_lex_vocab = [v.replace('_', ' ') for v in lit_lex_vocab]
lit_lex_vocab[:25]

['a[arm]n',
 'a[arm]n ak[do]v/t',
 'a[arm]n bad[open]v/t',
 'a[arm]n dar[split]v/t',
 'a[arm]n daŋal[wide]v/i',
 'a[arm]n durah[goat]n',
 'a[arm]n e[leave]v/i',
 'a[arm]n gab[left]n',
 'a[arm]n gal[big]v/i',
 'a[arm]n gud[ox]n',
 'a[arm]n gur[thick]v/i',
 'a[arm]n gur[turn]v/i',
 'a[arm]n il[raise]v/t',
 'a[arm]n kalag[strong]v/i',
 'a[arm]n kud[cut]v/t',
 'a[arm]n la[hang]v/t',
 'a[arm]n mah[great]v/i',
 'a[arm]n me[battle]n',
 'a[arm]n sag[good]v/i',
 'a[arm]n si[horn]n sa[equal]v/t',
 'a[arm]n sig[weak]v/i',
 'a[arm]n tal[broad]v/i',
 'a[arm]n tulu[slacken]v/t',
 'a[arm]n ud[sun]n',
 'a[arm]n zid[right]v/i']

### Document Term Matrix of Lexical Lines

The lexical corpus is transformed into a Document Term Matrix (or DTM), in the same way we did in [3.2](./3_2_Lit_Lex.ipynb) for the literary corpus - but with two important differences. 

First, the parameter `ngram_range` is set to (1, 5). With this parameter, `Countvectorizer()` will create a column for each word (ngram n=1), but also for each sequence of two words (bigram; n=2), or three words (trigram; n=3), etc. The entry `amar\[young\]n ga\[milk\]n gu\[eat\]v/t` ("calf that eats milk") will be represented as :

| type             | representation  |
|------------------|-----------------|
| unigram        | amar\[young\]n |
|                    | ga\[milk\]n |
|                    | gu\[eat\]v/t |
| bigram             | amar\[young\]n ga\[milk\]n |
|                    | ga\[milk\]n gu\[eat\]v/t |
| trigram | amar\[young\]n ga\[milk\]n gu\[eat\]v/t|

For longer entries we will also get 4-grams and 5-grams. 

A three word entry which was treated as a single unit in [3.1](./3_1_Lit_Lex_Vocab.ipynb) and [3.2](./3_2_Lit_Lex.ipynb) now results in 6 potential columns in the Document Term Matrix. 

Potentially, this results in a very big (and very sparse) matrix. In order to limit the its size somewhat we use the vocabulary `lit_lex_vocab` which contains all lemmas and lexical entries shared by the lexical and literary corpora. These are the relevant vocabulary items that we wish to explore.

Second, we will take as our data the column `lemma` from the DataFrame `lex_lines` which represents the lexical corpus in *line* format, rather than in document format. This is important, because we do not want the to jump over line boundaries. The DTM will thus treat each line as a document. In the next step we will add up all the line-based data to create a proper DTM.

In [8]:
cv = CountVectorizer(preprocessor = lambda x: x, tokenizer = lambda x: x.split(), vocabulary = lit_lex_vocab, ngram_range=(1, 5))
dtm = cv.fit_transform(lex_lines['lemma'])
lex_lines_dtm = pd.DataFrame(dtm.toarray(), columns= cv.get_feature_names(), index=lex_lines["id_text"])

### Aggregate DTM to Lexical Compositions
We may now reformat the line-based DTM into a true DTM by using the Pandas functions `groupby()` and `aggregate()`. The `aggregate()` function, in this case, is `sum`: for every word or ngram add the frequencies of all the lines of a single lexical composition.

Then the number of matches is computed. The field `n_matches` represents the number of unique words or ngrams that a lexical document shares with the literary corpus. For the code see [3.2](./3_2_Lit_Lex.ipynb) section 3.2.2.

In [9]:
lex_comp_dtm = lex_lines_dtm.groupby('id_text').agg(sum).reset_index()
lex_comp_dtm["n_matches"] = lex_comp_dtm[lit_lex_vocab].astype(bool).sum(axis = 1)
lex_comp_dtm

,id_text,a[arm]n,a[arm]n ak[do]v/t,a[arm]n bad[open]v/t,a[arm]n dar[split]v/t,a[arm]n daŋal[wide]v/i,a[arm]n durah[goat]n,a[arm]n e[leave]v/i,a[arm]n gab[left]n,a[arm]n gal[big]v/i,...,šutur[garment]n,šuziʾana[1]dn,šuš[cover]v/t,šušana[one-third]nu,šuši[sixty]nu,šušin[1]sn,šušru[distressed]v/i,šuʾi[barber]n,šuʾura[goose]n,n_matches
0,dcclt/P117394,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
1,dcclt/P117395,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,dcclt/P117396,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
3,dcclt/P117397,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,dcclt/P117404,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2133,dcclt/signlists/P447992,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
2134,dcclt/signlists/P447993,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9
2135,dcclt/signlists/P447994,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,96
2136,dcclt/signlists/P447997,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,40


# Remove Duplicates
Since the lexical data are drawn from multiple (sub)projects, it is possible that there are duplicates. Duplicates have the same P, Q, or X number. We select the version with the largest number of (lemmatized) words and drop others.

First we add the field `length` from the DataFrame `lex_comp` to the DataFrame `lex_comp_dtm` by merging on the field `id_text`. The merge method is `inner` (only merging those rows that are available in both DataFrames) so that documents that were omitted from `lex_comp` (because of length zero) do not show up again. Second, the field `id_text`, which has the format `dcclt/Q000041` or `dcclt/signlists/P447992`, is reduced to only the last 7 positions (P, Q, or X, followed by six digits). The merged DataFrame is ordered by length (from large to small) and, if duplicate `text_id`s are found, only the first one is kept with the Pandas method `drop_duplicates()`.

In [10]:
lex_comp_dtm = pd.merge(lex_comp_dtm, lex_comp[['id_text', 'length']], on = 'id_text', how = 'inner')
lex_comp_dtm['id_text'] = lex_comp_dtm['id_text'].str[-7:]
lex_comp_dtm = lex_comp_dtm.sort_values(by = 'length', ascending=False)
lex_comp_dtm = lex_comp_dtm.drop_duplicates(subset = 'id_text', keep = 'first')

### Adding Metadata
The metadata of the lexical texts (such as composition name, etc.) is found in the JSON files for each of the (sub)projects downloaded in section [3.1](./3_1_Lit_Lex_Vocab.ipynb). The code is essentially the same as in [3.2](./3_2_Lit_Lex.ipynb), but since there are multiple (sub)projects involved, it is done in a loop.

In [11]:
cat = {}
for proj in ['dcclt', 'dcclt/signlists', 'dcclt/nineveh', 'dcclt/ebla']:
    f = proj.replace('/', '-')
    file = f"jsonzip/{f}.zip" # The ZIP file was downloaded in notebook 3_1
    z = zipfile.ZipFile(file) 
    st = z.read(f"{proj}/catalogue.json").decode("utf-8")
    j = (json.loads(st))
    cat.update(j["members"])
cat_df = pd.DataFrame(cat).T
cat_df["id_text"] = cat_df["id_text"].fillna(cat_df["id_composite"])
cat_df = cat_df.fillna('')
cat_df = cat_df[["id_text", "designation", "subgenre"]]

### Merging
Now merge `cat_df` with the DataFrame `lex_comp_dtm` on the field `id_text`. Of the DTM we only keep the fields `n_matches` and `length`. The results are shown in descending order of the number of matches.

In [12]:
lex = pd.merge(cat_df, lex_comp_dtm[['id_text', 'n_matches', 'length']], on = 'id_text', how = 'inner')
lex.sort_values(by='n_matches', ascending = False)

,id_text,designation,subgenre,n_matches,length
763,Q000050,OB Nippur Izi,Acrographic Word Lists,688,1400
761,Q000047,OB Nippur Lu,Thematic Word Lists,608,1459
765,Q000055,OB Nippur Ea,Sign Lists,599,779
762,Q000048,OB Nippur Kagal,Acrographic Word Lists,447,1015
755,Q000039,OB Nippur Ura 01,Thematic Word Lists,445,1289
...,...,...,...,...,...
11,P209818,"Ontario 2, 502",OB Ura,0,3
54,P225122,"TIM 10, 114",OB Ugumu,0,3
43,P225059,"TIM 10, 083",exercise,0,3
3,P117397,"MVN 13, 624",,0,3


### Normalizing
Long lexical documents have more matches than short one. Normalize by dividing the number of matches (`n_matches`) by text length. For very short documents this measure has little value; only longer documents are displayed. Since the number of matches is based on *ngrams*, it is possible that `n_matches` is larger than `length` and that `norm` is higher than 1. This only happens in very short documents.

In [13]:
lex['norm'] = lex['n_matches'] / lex['length']
lex = lex.sort_values(by = 'norm', ascending = False)
lex.loc[lex.length > 250]

,id_text,designation,subgenre,n_matches,length,norm
178,P228842,"MSL 14, 018 Bb",OB Nippur Ea,333,410,0.812195
765,Q000055,OB Nippur Ea,Sign Lists,599,779,0.768935
766,Q000056,OB Nippur Aa,Sign Lists,231,408,0.566176
718,P447992,"OECT 04, 152",OB Diri Oxford,145,293,0.494881
763,Q000050,OB Nippur Izi,Acrographic Word Lists,688,1400,0.491429
418,P247810,IB 1514,OB Lu,133,274,0.485401
770,Q002268,OB Nippur Ugumu,Thematic Word Lists,163,348,0.468391
767,Q000057,OB Nippur Diri,Sign Lists,278,594,0.468013
764,Q000052,Nippur Nigga,Acrographic Word Lists,391,844,0.463270
762,Q000048,OB Nippur Kagal,Acrographic Word Lists,447,1015,0.440394


### Explore the Results
Explore the results in an interactive table.

In [14]:
anchor = '<a href="http://oracc.org/dcclt/{}", target="_blank">{}</a>'
lex2 = lex.copy()
lex2['id_text'] = [anchor.format(val,val) for val in lex['id_text']]
lex2['PQ'] = ['Composite' if i[0] == 'Q' else 'Exemplar' for i in lex['id_text']]

In [15]:
@interact(sort_by = lex2.columns, rows = (1, len(lex2), 1), min_length = (0,500,5), show = ["Exemplars", "Composites", "All"])
def sort_df(sort_by = "norm", ascending = False, rows = 25, min_length = 250, show = 'All'):
    if not show == 'All':
        l = lex2.loc[lex2['PQ'] == show[:-1]]
    else:
        l = lex2
    l = l.drop('PQ', axis = 1)
    l = l.loc[l.length >= min_length].sort_values(by = sort_by, ascending = ascending).reset_index(drop=True)[:rows].style
    return l

interactive(children=(Dropdown(description='sort_by', index=5, options=('id_text', 'designation', 'subgenre', …

### Discussion
When ordered by `norm` the top of the list is formed by lexical compositions such as the sign lists [OB Nippur Ea](http://oracc.org/dcclt/Q000055) and [OB Nippur Diri](http://oracc.org/dcclt/Q000057), the acrographic list/list of professions [OB Nippur Lu](http://oracc.org/dcclt/Q000047), and the acrographic lists [OB Nippur Izi](http://oracc.org/dcclt/Q000050), and [OB Nippur Kagal](http://oracc.org/dcclt/Q000048), or (large) exemplars of such compositions. If we restrict the DataFrame to composites (Q numbers) only, this comes out even clearer. All these lexical texts belong to what Jay Crisostomo has labeled "ALE": Advanced Lexical Exercises ([Translation as Scholarship](https://doi.org/10.1515/9781501509810); SANER 22, 2019). These exercises belong to the advanced first stage of education, just before students would start copying literary texts. The thematic lists collected in [Ura](http://oracc.org/dcclt/Q000039,Q000040,Q000001,Q000041,Q000042,Q000043) (lists of trees, wooden objects, reeds, reed objects, clay, pottery, hides, metals and metal objects, animals, meat cuts, fish, birds, plants, etc.) have much lower `norm` values and thus less overlap with literary vocabulary.

### Important Words: TF-IDF
What the above analysis does not take into aacount is the significance of the matching words. Words such as **igi\[eye\]n** or **šag\[heart\]n** are used in many kinds of expressions and will be found in all literary compositions, except for the most fragmentary ones. Those words are matched by the list [Ugumu](http://oracc.org/dcclt/Q002268) (list of body parts) but such matches are much less meaningful than a rarer word or expression like **nir\[stone\]n babbardil\[~stone\]n** (a type of agate with a single white stripe) which is fairly common in the lexical tradition, but only appears in one place in the literary corpus ([UET 6/3 561](http://oracc.org/epsd2/literary/P346598)) and with some frequency in administrative documents.

In order to distinguish between more and less significant matches we will compute the [tf-idf](https://en.wikipedia.org/wiki/Tf-idf) value for each word or expression in each literary document. Tf-idf is short for term frequency - inverse document frequency and is a widely used measure (or rather a class of measures) for weighing the importance of a word in a document. Term frequency refers to the frequency of a term in a document; a term that is more frequently used is probably important in that document. Document frequency refers to the number of documents in the corpus in which the term is found. A term that is found in the great majority of documents has little distinguishing value. A high document frequency, therefore, results in a lower relevance of the term. Thus the tf-idf value of words like **igi\[eye\]n** or **šag\[heart\]n** in any document, even though they may appear very frequently, is expected to be low because they appear in almost every document. On the other hand, the tf-idf value of **nir\[stone\]n babbardil\[~stone\]n** in [UET 6/3 561](http://oracc.org/epsd2/literary/P346598) is going to be fairly low, too, because the word appears only once in that text. A high value is associated with high frequency in a few documents. 

We can compute tf-idf values for the literary corpus with `TfidfVectorizer()` from the `sklearn` package. `TfidfVectorizer()` is a close relative of `CountVectorizer()` and works in essentially the same way.

> # Note: 
This is probably not a good idea. It is too complex, with computing means, adding up means and taking the mean of means - not a good way to introduce tf-idf. Probably better to explore in more detail the kinds of words that are matched.

In [16]:
lit_lines = pd.read_pickle('output/litlines.p')
lit_comp = lit_lines.groupby(['id_text']).agg({'lemma' : ' '.join}).reset_index()
lit_comp['id_text'] = lit_comp['id_text'].str[-7:]
tv = TfidfVectorizer(tokenizer = lambda x: x.split(), preprocessor = lambda x: x, ngram_range = (1,5), vocabulary =lit_lex_vocab)
dtm = tv.fit_transform(lit_comp['lemma'])
lit_df = pd.DataFrame(dtm.toarray(), columns= tv.get_feature_names(), index=lit_comp["id_text"])
lit_df

,a[arm]n,a[arm]n ak[do]v/t,a[arm]n bad[open]v/t,a[arm]n dar[split]v/t,a[arm]n daŋal[wide]v/i,a[arm]n durah[goat]n,a[arm]n e[leave]v/i,a[arm]n gab[left]n,a[arm]n gal[big]v/i,a[arm]n gud[ox]n,...,šutum[storehouse]n,šutur[garment]n,šuziʾana[1]dn,šuš[cover]v/t,šušana[one-third]nu,šuši[sixty]nu,šušin[1]sn,šušru[distressed]v/i,šuʾi[barber]n,šuʾura[goose]n
id_text,,,,,,,,,,,,,,,,,,,,,
P209784,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
P251427,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
P251713,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
P251728,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
P252215,0.070647,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Q000823,0.043850,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.010059,0.0,0.0,0.0,0.0,0.0,0.0
Q000824,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
Q000825,0.048206,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.044233,0.0,0.0,0.0,0.0,0.0,0.0


### Mean tf-idf weight
We can compute the mean tf-idf value of each word by dividing the sum of each column (the total tf-idf weights for a word or expression) by the number of rows in the DTM. This results in an array of values for each column in the DTM.

In [17]:
mean = lit_df[lit_lex_vocab].values.mean(axis=0)
mean

array([2.22982571e-02, 1.16028618e-04, 8.45282174e-04, ...,
       2.10628298e-04, 6.07434282e-04, 6.65925796e-05])

The array `mean` represents the mean weight of each word or expression in the literary corpus. We can multiply the entire lexical DTM with that array.

In [18]:
lit_lex_tfidf = lex_comp_dtm
lit_lex_tfidf[lit_lex_vocab] = lit_lex_tfidf[lit_lex_vocab].astype(bool).mul(mean, axis = 1)

In [20]:
lit_lex_tfidf['weighted'] = lit_lex_tfidf[lit_lex_vocab].sum(axis=1)

In [21]:
lit_lex_tfidf

,id_text,a[arm]n,a[arm]n ak[do]v/t,a[arm]n bad[open]v/t,a[arm]n dar[split]v/t,a[arm]n daŋal[wide]v/i,a[arm]n durah[goat]n,a[arm]n e[leave]v/i,a[arm]n gab[left]n,a[arm]n gal[big]v/i,...,šuš[cover]v/t,šušana[one-third]nu,šuši[sixty]nu,šušin[1]sn,šušru[distressed]v/i,šuʾi[barber]n,šuʾura[goose]n,n_matches,length,weighted
715,Q000043,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.000607,0.0,347,2542,1.560452
717,Q000047,0.022298,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000499,0.000000,...,0.007067,0.000000,0.0,0.0,0.0,0.000607,0.0,608,1459,2.425656
719,Q000050,0.022298,0.000116,0.000845,0.000238,0.000161,0.000238,0.0,0.000499,0.001499,...,0.007067,0.000358,0.0,0.0,0.0,0.000607,0.0,688,1400,2.765051
711,Q000039,0.022298,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000358,0.0,0.0,0.0,0.000607,0.0,445,1289,1.575015
712,Q000040,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.007067,0.000358,0.0,0.0,0.0,0.000607,0.0,409,1182,1.423768
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
567,P333845,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,1,1,0.000169
755,P333147,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,1,1,0.000218
753,P333145,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0,1,0.000000
591,P346867,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,1,1,0.000519


In [32]:
lex2 = pd.merge(cat_df, lit_lex_tfidf[['id_text', 'weighted', 'length', 'n_matches']], on = 'id_text', how = 'inner')
#lex2 = pd.merge(lex2, lex[['length', 'n_matches', 'id_text']], on = 'id_text', how = 'inner')

Instead of dividing by length look at mean value of weighted
```python
lex2['norm'] = lex2['weigthed'] / lex2.astype(bool).sum(axis = 1)
```

In [33]:
#lex2['norm'] = lex2['weighted'] / lex2['n_matches']
lex2['tfidfnorm'] = lex2['weighted'] / lex2['length']
lex2.loc[lex2.length >= 250].sort_values(by = 'tfidfnorm', ascending = False)

,id_text,designation,subgenre,weighted,length,n_matches,tfidfnorm
178,P228842,"MSL 14, 018 Bb",OB Nippur Ea,1.825684,410,333,0.004453
765,Q000055,OB Nippur Ea,Sign Lists,2.922156,779,599,0.003751
418,P247810,IB 1514,OB Lu,0.749955,274,133,0.002737
766,Q000056,OB Nippur Aa,Sign Lists,1.089940,408,231,0.002671
770,Q002268,OB Nippur Ugumu,Thematic Word Lists,0.821388,348,163,0.002360
561,P310404,YBC 09868,,1.242509,538,221,0.002309
762,Q000048,OB Nippur Kagal,Acrographic Word Lists,2.264700,1015,447,0.002231
556,P300937,NBC 09830,,0.849935,408,157,0.002083
721,P447996,Anonymous 447996,OB Ura,0.669135,324,113,0.002065
387,P230258,A 07896,OB Ura,0.536252,260,109,0.002063


### Explore
Explore the results in an interactive table.

In [37]:
anchor = '<a href="http://oracc.org/dcclt/{}", target="_blank">{}</a>'
lex3 = lex2.copy()
lex3['id_text'] = [anchor.format(val,val) for val in lex2['id_text']]
lex3['PQ'] = ['Composite' if i[0] == 'Q' else 'Exemplar' for i in lex2['id_text']]

In [39]:
@interact(sort_by = lex3.columns, rows = (1, len(lex2), 1), min_length = (0,500,5), show = ["Exemplars", "Composites", "All"])
def sort_df(sort_by = "tfidfnorm", ascending = False, rows = 25, min_length = 250, show = 'All'):
    if not show == 'All':
        l = lex3.loc[lex3['PQ'] == show[:-1]]
    else:
        l = lex3
    l = l.drop('PQ', axis = 1)
    l = l.loc[l.length >= min_length].sort_values(by = sort_by, ascending = ascending).reset_index(drop=True)[:rows].style
    return l

interactive(children=(Dropdown(description='sort_by', index=6, options=('id_text', 'designation', 'subgenre', …